In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 1000)
from utils import get_engine

engine = get_engine()

## Code pregnancy end hospitalizations

The first part of this contains SAS code references copied from the FAD SMM methodology, translated to Pandas.

Source: https://www.documentcloud.org/documents/25539312-hrsa-federally-available-data-fad-resource-document-for-fy25fy23-application-annual-report/

### Code delivery/abortive events (HRSA algorithm)

In [ ]:
delivery_v27 = pd.read_sql(
"""
select record_id from discharge_diagnosis
where code like 'Z37%%';
""",con=engine
).drop_duplicates(subset='record_id')

delivery_v27['delivery_v27'] = 1
delivery_v27.head()

In [ ]:
# All of these codes are categorized as a "normal delivery"; some are c-sections
normal_delivery = pd.read_sql(
"""
select record_id,code from discharge_diagnosis
where code in ('O80','O82','O7582');
""",con=engine
).drop_duplicates()

normal_delivery['present'] = 1
normal_delivery = normal_delivery.pivot(
    index='record_id',columns='code',values='present'
).reset_index().fillna(False)

normal_delivery['delivery_650'] = 1

# identify c-sections
normal_delivery['cesarean_dx'] = normal_delivery[['O82','O7582']].eq(1).any(axis=1)
normal_delivery.cesarean_dx = normal_delivery.cesarean_dx.replace(True,1).replace(False,0)

normal_delivery = normal_delivery.drop(['O80','O82','O7582'],axis=1)

normal_delivery.head()

In [ ]:
# These are the top-level codes but there are tons of 
# sub-codes so has to be a partial string search
abortive_outcome = pd.read_sql(
"""
select record_id from discharge_diagnosis
where code like any (array['O00%%','O01%%','O02%%','O03%%','O04%%','O07%%','O08%%']);
""",con=engine
).drop_duplicates(subset="record_id")

abortive_outcome['abort_dx'] = 1

abortive_outcome.head()

In [ ]:

delivery_procedures = pd.read_sql("""
    select record_id,code from discharge_procedure
        where (
            (code >= '10D00Z0' and code <= '10D00Z2')
            or
            (code >= '10D07Z3' AND code <= '10D07Z8')
            or
            (code = '10E0XZZ')
            )
""",con=engine).drop_duplicates()

delivery_procedures['present'] = 1
delivery_procedures = delivery_procedures.pivot(
    index="record_id",columns='code',values="present"
).reset_index()
delivery_procedures = delivery_procedures.fillna(0)
delivery_procedures['delivery_pr'] = 1

delivery_procedures['cesarean_pr'] = delivery_procedures[['10D00Z0','10D00Z1','10D00Z2']].eq(1).any(axis=1)
delivery_procedures.cesarean_pr = delivery_procedures.cesarean_pr.replace(True,1).replace(False,0)

delivery_procedures = delivery_procedures.drop([
    '10D00Z0', '10D00Z1', '10D00Z2', '10D07Z3', '10D07Z4',
    '10D07Z5', '10D07Z6', '10D07Z7', '10D07Z8', '10E0XZZ'
],axis=1)

delivery_procedures.head()

In [ ]:
abortive_outcome_pr = pd.read_sql(
"""
select record_id from discharge_procedure
where code like '10A0%%';
""",con=engine
).drop_duplicates(subset="record_id")

abortive_outcome_pr['abort_pr'] = 1

abortive_outcome_pr.head()

In [ ]:
delivery_drg = pd.read_sql("""
    select record_id, frozen_ms_drg from discharge
    where (
    (frozen_ms_drg >= '765' and frozen_ms_drg <= '768')
    or
    (frozen_ms_drg in ('774','775'))
    or
    (frozen_ms_drg >= '783' and frozen_ms_drg <= '788')
    or
    (frozen_ms_drg >= '796' and frozen_ms_drg <= '798')
    or
    (frozen_ms_drg >= '805' and frozen_ms_drg <= '807')
    )
""",con=engine)

# don't need to pivot this because each stay has only 1 drg value
delivery_drg['delivery_drg'] = 1
delivery_drg['cesarean_drg'] = np.where(
    (
        delivery_drg.frozen_ms_drg.isin([
            '765','766',
            '783','784','785','786','787','788'
,            ])
    ),1,0
)
delivery_drg = delivery_drg.drop(['frozen_ms_drg'],axis=1)

delivery_drg.head()

Combine all coded rows into one dataframe with all births and abortive outcomes

In [ ]:
delivery = pd.merge(delivery_v27,normal_delivery,how="outer",on="record_id")
delivery = pd.merge(delivery,delivery_procedures,how='outer',on="record_id")
delivery = pd.merge(delivery,delivery_drg,how="outer",on='record_id')

delivery = pd.merge(delivery,abortive_outcome,how="outer",on="record_id")
delivery = pd.merge(delivery,abortive_outcome_pr,how='outer',on="record_id")

delivery = delivery.fillna(0)

delivery_year = pd.read_sql("""
    select record_id, discharge,year
        from discharge
    where record_id in {0}
""".format(tuple(delivery.record_id.unique())),con=engine)

delivery = pd.merge(
    delivery_year,delivery,
    how="outer",on="record_id"
)
delivery.head()

In [ ]:
# all abortive outcomes (i.e. ectopic, miscarriage, abortion)
delivery['abort'] = np.where(
    (delivery.abort_dx.eq(1)|delivery.abort_pr.eq(1)),
    1,0
)

# all births
delivery['birth'] = np.where(
    (
        (delivery.delivery_v27.eq(1)|delivery.delivery_650.eq(1)|
        delivery.delivery_drg.eq(1)|delivery.delivery_pr.eq(1)|delivery.cesarean_dx.eq(1)) &
        delivery.abort.eq(0)
    ),1,0
)

# all cesarean codes
delivery['cesarean'] = np.where(
    (delivery.cesarean_dx.eq(1)|\
     delivery.cesarean_pr.eq(1)|\
     delivery.cesarean_drg.eq(1)),
    1,0
)

delivery[delivery.birth.eq(1)].year.value_counts(sort=False)

In [ ]:
# list record_ids so we can narrow our next queries
# to just rows we've identified as pregnancy-ending hospitalizations.
record_ids = delivery.record_id.unique()

In [ ]:
# create col that is true for both births and abortive outcomes.
# This is all hospitalizations with the end of a pregnancy
delivery['pregnancy_end'] = 1

In [ ]:
# clean up dataframe columns
delivery = delivery.drop([
    'delivery_v27','delivery_650','delivery_pr','delivery_drg',
    'abort_dx','abort_pr','cesarean_dx','cesarean_pr','cesarean_drg'
],axis=1)

delivery.head()

### Code SMM (HRSA algorithm)

In [ ]:
smm1 = pd.read_sql("""
    select record_id from discharge_diagnosis
        where code like any (array['I21%%','I22%%'])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm1['smm1'] = 1

In [ ]:
smm2 = pd.read_sql("""
    select record_id from discharge_diagnosis
        where code like any (array['I71%%','I790%%'])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm2['smm2'] = 1

In [ ]:
smm3 = pd.read_sql("""
    select record_id from discharge_diagnosis
        where code like any (array['N17%%','O904%%'])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm3['smm3'] = 1

In [ ]:
smm4 = pd.read_sql("""
    select record_id from discharge_diagnosis
        where code like any (array[
        'J80%%','J951%%','J952%%','J953%%','J9582%%','J960%%',
        'J962%%','J969%%','R0603%%','R092%%'
])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm4['smm4'] = 1

In [ ]:
smm5 = pd.read_sql("""
    select record_id from discharge_diagnosis
        where code like any (array[
        'O88112%%','O88113%%','O88119%%','O8812%%','O8813%%'
])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm5['smm5'] = 1

In [ ]:
smm6 = pd.read_sql("""
    select record_id from discharge_diagnosis
        where code like any (array[
        'I46%%','I490%%'
])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm6['smm6'] = 1

In [ ]:
smm8 = pd.read_sql("""
    select record_id from discharge_diagnosis
        where code like any (array[
        'D65%%','D688%%','D689%%',
        'O45002%%','O45003%%','O45009%%',
        'O45012%%','O45013%%','O45019%%',
        'O45022%%','O45023%%','O45029%%',
        'O45092%%','O45093%%','O45099%%',
        'O46002%%','O46003%%','O46009%%',
        'O46012%%','O46013%%','O46019%%',
        'O46022%%','O46023%%','O46029%%',
        'O46092%%','O46093%%','O46099%%',
        'O670%%','O723%%'
])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm8['smm8'] = 1

In [ ]:
smm9 = pd.read_sql("""
    select record_id from discharge_diagnosis
        where code like any (array[
        'O15%%'
])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm9['smm9'] = 1

In [ ]:
smm10 = pd.read_sql("""
    select record_id from discharge_diagnosis
        where code like any (array[
        'I9712%%','I9713%%','I9771%%'
])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm10['smm10'] = 1

In [ ]:
smm11 = pd.read_sql("""
    select record_id from discharge_diagnosis
        where code like any (array[
            'A812%%','G45%%','G46%%','G9349%%','H340%%','I60%%','I61%%','I62%%',
            'I6300%%',
            'I6301%%','I631%%', 'I632%%','I633%%','I634%%','I635%%','I636%%','I638%%',
            'I639%%','I65%%','I66%%','I67%%','I68%%','O2250%%','O2252%%','O2253%%',
            'I9781%%','I9782%%','O873%%'
])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm11['smm11'] = 1

In [ ]:
# NOTE: SMM12 is repeated for acute heart failure!
# I'm creating a separate variable so that we don't risk aggregating improperly

smm12a = pd.read_sql("""
    select record_id from discharge_diagnosis
        where code like any (array[
        'J810%%'
])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm12a['smm12a'] = 1

In [ ]:
smm12 = pd.read_sql("""
    select record_id from discharge_diagnosis
        where code like any (array[
        'I501%%','I5020%%','I5021%%','I5023%%','I5030%%','I5031%%','I5033%%',
        'I5040%%','I5041%%','I5043%%','I50810%%','I50811%%','I50813%%','I50814%%',
        'I5082%%','I5083%%','I5084%%','I5089%%','I509%%'
])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm12['smm12'] = 1

In [ ]:
smm13 = pd.read_sql("""
    select record_id from discharge_diagnosis
        where code like any (array[
        'O29112%%','O29113%%','O29119%%',
        'O29122%%','O29123%%','O29129%%',
        'O29192%%','O29193%%','O29199%%',
        'O29212%%','O29213%%','O29219%%',
        'O29292%%','O29293%%','O29299%%',
        'O740%%','O741%%','O742%%','O743%%',
        'O8901%%','O8909%%',
        'O891%%','O892%%',
        'T882XXA%%','T883XXA%%'
])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm13['smm13'] = 1

In [ ]:
smm14 = pd.read_sql("""
    select record_id from discharge_diagnosis
        where code like any (array[
        'A327%%','A40%%','A41%%','I76%%','O85%%',
        'O8604%%','R6520%%','R6521%%',
        'T8112XA%%','T8144XA%%'
])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm14['smm14'] = 1

In [ ]:
smm15 = pd.read_sql("""
    select record_id from discharge_diagnosis
        where code like any (array[
        'O751%%','R57%%',
        'T782XXA%%','T8110XA%%','T8111XA%%','T8119XA%%','T886XXA%%'
])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm15['smm15'] = 1

In [ ]:
smm16 = pd.read_sql("""
    select record_id from discharge_diagnosis
        where code like any (array[
        'D5700%%','D5701%%','D5702%%',
        'D57211%%','D57212%%','D57219%%',
        'D57411%%','D57412%%','D57419%%',
        'D57811%%','D57812%%','D57819%%'
])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm16['smm16'] = 1

In [ ]:
smm17 = pd.read_sql("""
    select record_id from discharge_diagnosis
        where code like any (array[
        'I2601%%','I2602%%','I2609%%',
        'I2690%%','I2692%%','I2693%%','I2694%%','I2699%%',
        'O88012%%','O88013%%','O88019%%',
        'O8802%%','O8803%%',
        'O88212%%','O88213%%','O88219%%',
        'O8822%%','O8823%%',
        'O88312%%','O88313%%','O88319%%',
        'O8832%%','O8833%%',
        'O88812%%','O88813%%','O88819%%',
        'O8882%%','O8883%%',
        'T800XXA%%'
])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm17['smm17'] = 1

In [ ]:
smm7 = pd.read_sql("""
    select record_id from discharge_procedure
    where code like any (array[
    '5A12012%%','5A2204Z%%'
    ])
        and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm7['smm7'] = 1

Note on smm18, blood transfusions - the SAS code has ICD9 but no ICD10 definition for this. [CDC](https://www.cdc.gov/maternal-infant-health/php/severe-maternal-morbidity/icd.html) has a list of ICD-10 codes for transfusion, but we're not using it for this analysis.

In [ ]:
smm19 = pd.read_sql("""
    select record_id from discharge_procedure
        where code like any (array[
        '0UT90ZL%%','0UT90ZZ%%','0UT97ZL%%','0UT97ZZ%%'
])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm19['smm19'] = 1

In [ ]:
smm20 = pd.read_sql("""
    select record_id from discharge_procedure
        where code like any (array[
    '0B110F4%%','0B113F4%%','0B114F4%%'
])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm20['smm20'] = 1

In [ ]:
smm21 = pd.read_sql("""
    select record_id from discharge_procedure
        where code like any (array[
        '5A1935Z%%', '5A1945Z%%', '5A1955Z%%'
])
            and record_id in {0};
""".format(tuple(record_ids)),con=engine).drop_duplicates()

smm21['smm21'] = 1


Join all the SMM columns

In [ ]:
smm = pd.merge(smm1,smm2,how="outer",on="record_id")
smm = pd.merge(smm,smm3,how="outer",on="record_id")
smm = pd.merge(smm,smm4,how="outer",on="record_id")
smm = pd.merge(smm,smm5,how="outer",on="record_id")
smm = pd.merge(smm,smm6,how="outer",on="record_id")
smm = pd.merge(smm,smm7,how="outer",on="record_id")
smm = pd.merge(smm,smm8,how="outer",on="record_id")
smm = pd.merge(smm,smm9,how="outer",on="record_id")
smm = pd.merge(smm,smm10,how="outer",on="record_id")
smm = pd.merge(smm,smm11,how="outer",on="record_id")
smm = pd.merge(smm,smm12a,how="outer",on="record_id")
smm = pd.merge(smm,smm12,how="outer",on="record_id")
smm = pd.merge(smm,smm13,how="outer",on="record_id")
smm = pd.merge(smm,smm14,how="outer",on="record_id")
smm = pd.merge(smm,smm15,how="outer",on="record_id")
smm = pd.merge(smm,smm16,how="outer",on="record_id")
smm = pd.merge(smm,smm17,how="outer",on="record_id")
smm = pd.merge(smm,smm19,how="outer",on="record_id")
smm = pd.merge(smm,smm20,how="outer",on="record_id")
smm = pd.merge(smm,smm21,how="outer",on="record_id")

smm = smm.fillna(0)
# if any smm types are true, then this variable is true
smm['smm'] = 1
# move the general smm variable to the beginning of the df
smm.insert(1,'smm',smm.pop('smm'))

Group the SMM columns

In [ ]:
smm['smm_hemorrhage'] = np.where(
    (smm.smm8.eq(1)|smm.smm15.eq(1)|smm.smm19.eq(1)),
    1,0
)

smm['smm_respiratory'] = np.where(
    (smm.smm4.eq(1)|smm.smm20.eq(1)|smm.smm21.eq(1)),
    1,0
)

smm['smm_cardiac'] = np.where(
    (smm.smm1.eq(1)|smm.smm2.eq(1)|smm.smm6.eq(1)|smm.smm7.eq(1)|smm.smm10.eq(1)|smm.smm12.eq(1)|smm.smm12a.eq(1)),
    1,0
)

smm['smm_renal'] = np.where(
    (smm.smm3.eq(1)),
    1,0
)

smm['smm_sepsis'] = np.where(
    (smm.smm14.eq(1)),
    1,0
)

smm['smm_hysterectomy'] = np.where(
    (smm.smm19.eq(1)),
    1,0
)

smm['smm_other_obstetric'] = np.where(
    (smm.smm5.eq(1)|smm.smm9.eq(1)|smm.smm13.eq(1)|smm.smm17.eq(1)),
    1,0
)

smm['smm_other_medical'] = np.where(
    (smm.smm11.eq(1)|smm.smm16.eq(1)),
    1,0
)

smm = smm.fillna(0)

In [ ]:
delivery_smm = pd.merge(delivery,smm,on="record_id",how="left")
delivery_smm = delivery_smm.fillna(0)
delivery_smm.head()

In [ ]:
# drop smm interstitial columns
delivery_smm = delivery_smm.drop([ 'smm1',
       'smm2', 'smm3', 'smm4', 'smm5', 'smm6', 'smm7', 'smm8', 'smm9', 'smm10',
       'smm11', 'smm12a', 'smm12', 'smm13', 'smm14', 'smm15', 'smm16', 'smm17',
       'smm19', 'smm20', 'smm21'
       ],axis=1)

### Age/gender filter (HRSA algorithm)
Identify rows with female patients with ages between 10 and 54.

In [ ]:
demographics = pd.read_sql(
"""
select record_id, pat_age, sex_code from discharge where record_id in {0}
""".format(tuple(record_ids)),con=engine
)

In [ ]:
delivery_female_age = demographics.copy()[demographics.sex_code.eq("F") & demographics.pat_age.isin([
    '04','05','06','07','08','09','10','11','12','13'    
])]

print("Unfiltered length:")
print(len(demographics))
print("Filtered length:")
print(len(delivery_female_age))

In [ ]:
delivery_female_age = delivery_female_age[['record_id']]
delivery_female_age['age_gender_filter'] = 1

delivery_smm.drop('age_gender_filter',axis=1,inplace=True,errors="ignore")
delivery_smm = pd.merge(delivery_smm,delivery_female_age,on="record_id",how="left")
delivery_smm.age_gender_filter = delivery_smm.age_gender_filter.fillna(0)

In [ ]:
delivery_smm.head()

### Weeks gestation

We query all weeks gestation codes, then translate those to an integer. To account for long hospitalizations with multiple gestational week codes, we drop all but the highest week code for each hospitalization to indicate the gestational week at the time the pregnancy ended.

In [ ]:
weeks_gestation = pd.read_sql("""
        select record_id, code from discharge_diagnosis
            where code like 'Z3A%%' and record_id in {0}
    """.format(tuple(record_ids)),con=engine)
weeks_gestation['weeks_gestation'] = weeks_gestation.code.str.replace("Z3A","")
# exclude gestational week unknown
weeks_gestation = weeks_gestation[~weeks_gestation.weeks_gestation.eq("00")]
weeks_gestation.weeks_gestation = weeks_gestation.weeks_gestation.astype(int)

weeks_gestation = weeks_gestation[['record_id','weeks_gestation']].sort_values("weeks_gestation").drop_duplicates(subset='record_id',keep='last')

In [ ]:
delivery_smm = pd.merge(delivery_smm,weeks_gestation[['record_id','weeks_gestation']],on="record_id",how="left")

delivery_smm.head()

### Abortive outcomes and complications
These diagnoses are related to "abortive outcomes", i.e. miscarriage and abortion events

In [ ]:
abortive = pd.read_sql("""
        select record_id,code,present_on_admission from discharge_diagnosis
                where record_id in {0}
                and code like any (array['O00%%','O01%%','O02%%','O03%%','O04%%','O07%%','O08%%']);
    """.format(tuple(delivery_smm.record_id)),con=engine).drop_duplicates()

Identify abortive sepsis codes

In [ ]:
abortive_sepsis = abortive.copy()[abortive.code.str.startswith(('O0337','O0387','O0487','O0737','O0882'))][['record_id']].drop_duplicates()
abortive_sepsis['abortive_sepsis'] = 1

abortive_sepsis.head()

In [ ]:
delivery_smm = pd.merge(delivery_smm,abortive_sepsis,on="record_id",how="left")
delivery_smm = pd.merge(delivery_smm,missed_abortion,on="record_id",how="left")
delivery_smm = pd.merge(delivery_smm,missed_abortion_admission,on="record_id",how="left")
delivery_smm.abortive_sepsis = delivery_smm.abortive_sepsis.fillna(0)
delivery_smm.missed_abortion = delivery_smm.missed_abortion.fillna(0)
delivery_smm.missed_abortion_admission = delivery_smm.missed_abortion_admission.fillna(0)
delivery_smm.head()

Define composite sepsis column, which includes SMM sepsis and abortive sepsis.

In [ ]:
delivery_smm.loc[(delivery_smm.smm_sepsis.eq(1)|delivery_smm.abortive_sepsis.eq(1)),'sepsis_combined'] = 1
delivery_smm.sepsis_combined = delivery_smm.sepsis_combined.fillna(0)

delivery_smm.head()

### Missed abortion
Indicates fetal death in utero before 20 weeks

In [ ]:
# identify all rows with a missed abortion code
missed_abortion = abortive.copy()[abortive.code.eq('O021')]
missed_abortion = missed_abortion[['record_id']].drop_duplicates()
missed_abortion['missed_abortion'] = 1

# for missed abortion codes, identify ones marked "present on admission"
missed_abortion_admission = abortive.copy()[abortive.code.eq('O021') & abortive.present_on_admission.eq("Y")][['record_id']].drop_duplicates()
missed_abortion_admission['missed_abortion_admission'] = 1

### Intrauterine fetal demise
Indicates fetal death in utero at 20 weeks and up

In [ ]:
iufd_codes = pd.read_sql("""
    select * from discharge_diagnosis
        where record_id in {0}
        and code like 'O364%%';
""".format(tuple(delivery_smm.record_id)),con=engine)

iufd_codes

In [ ]:
iufd_codes['iufd'] = 1
iufd_codes.loc[iufd_codes.present_on_admission.eq("Y"),'iufd_on_admission'] = 1

on_admission = iufd_codes[iufd_codes.iufd_on_admission.eq(1)][['record_id','iufd_on_admission']].drop_duplicates()
iufd = iufd_codes[['record_id','iufd']].drop_duplicates()
delivery_smm.drop([c for c in delivery_smm.columns if c.find("iufd")==0],axis=1,inplace=True)
delivery_smm = pd.merge(delivery_smm,iufd,on="record_id",how="left")
delivery_smm = pd.merge(delivery_smm,on_admission,on="record_id",how="left")

In [ ]:
delivery_smm.iufd_on_admission.value_counts()

In [ ]:
delivery_smm.iufd = delivery_smm.iufd.fillna(0)
delivery_smm.iufd_on_admission = delivery_smm.iufd_on_admission.fillna(0)
delivery_smm.head()

### Combine IUFD and missed abortion
This gives us our fetal demise on admission definition

In [ ]:
delivery_smm.loc[
    delivery_smm.iufd_on_admission.eq(1)|delivery_smm.missed_abortion_admission.eq(1),
    'fetal_demise_admission'] = 1

delivery_smm.fetal_demise_admission = delivery_smm.fetal_demise_admission.fillna(0)


### Save to file

In [ ]:
delivery_smm.to_csv("../data/processed/delivery_smm.csv",index=False)

In [ ]:
delivery_smm.record_id.value_counts()

In [ ]:
delivery_smm.year.value_counts()